# Flight Recorder Analysis Demo

Load recorded ticks, orders, and fills into pandas for transaction cost analysis.

Update `DATA_DIR` to point at the `data/flight_recorder` directory generated by `tesser-cli`.

In [ ]:
from pathlib import Path

import pandas as pd

from tesser.analysis.execution import calculate_slippage, summarize_fills
from tesser.data import DataStore, load_fills, load_orders, load_ticks

In [ ]:
DATA_DIR = Path("../../data/flight_recorder")
store = DataStore(DATA_DIR)
window = dict(symbol="BTCUSDT", start="2024-01-01", end="2024-01-07")
orders = load_orders(store, **window)
fills = load_fills(store, **window)
ticks = load_ticks(store, **window)
print(f"Loaded {len(orders)} orders, {len(fills)} fills, {len(ticks)} ticks.")

In [ ]:
slippage = calculate_slippage(orders, fills, ticks_df=ticks)
slippage.sort_values("created_at").head()

In [ ]:
fills_with_algo = fills.merge(
    slippage[["id", "algo_label"]], left_on="order_id", right_on="id", how="left"
)
algo_summary = summarize_fills(fills_with_algo, group_by=("algo_label",))
algo_summary

In [ ]:
ax = slippage.set_index("created_at")["slippage_bps"].dropna().plot(
    kind="bar", figsize=(10, 4), title="Implementation Shortfall (bps)"
)
ax.set_ylabel("bps")